In [1]:
# _*_ coding:utf-8 _*_
'''
部门人员情况
'''
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
from pylab import mpl
import datetime
import docx


def df_age(df1):
    df=pd.to_datetime(df1)
    now_year=datetime.datetime.today().year
    df_date=now_year-df.datetime.year
    return df_date

mpl.rcParams['font.sans-serif']=['Microsoft YaHei']

db= pymysql.connect(host='localhost',
                   port=3306,
                   user='root',
                   passwd='kongling8167',
                   db='kmsf',
                   charset='utf8'
                   )
sql="SELECT * FROM 安监部 WHERE 岗位状态 !='调离'"
df=pd.read_sql(sql,db)

#df['年龄']=df_age(pd.DataFrame(df['出生日期']))

df_count=df['工作岗位'].value_counts()
sql="SELECT * FROM 岗位设置表"
df_post=pd.read_sql(sql,db)





df_bor=df[df['岗位状态']=='借出']
df_bor_count=df_bor['工作岗位'].value_counts()

df_post_count=df_post['岗位名称'].value_counts()
df_post_plus=pd.merge(df_count,df_post_count,left_index=True,right_index=True)
df_post_plus=pd.merge(df_post_plus,df_bor_count,how='left',left_index=True,right_index=True )
df_post_plus=df_post_plus.reset_index()
df_post_plus.columns=['岗位','在岗','定员','外借']
df_post_plus['缺员']=df_post_plus['定员']-df_post_plus['在岗']
df_post_plus=df_post_plus.fillna(0)
df_post_plus['外借']=df_post_plus['外借'].astype('int')

sum=df_post_plus['在岗'].sum()

#print(df_count,'\n',df_post_count)
print(df_post_plus)
text="安全监管部定员{0}人，实际在岗{1}人，外借{2}人。".format(df_post_plus['定员'].sum(),df_post_plus['在岗'].sum(),df_post_plus['外借'].sum())

print(text,'外借人员为：')
print(df_bor[['工作岗位','姓名']])

df_count_degree=df['文化程度'].value_counts().reset_index()
df_count_degree.columns=['文化程度','数量']
print('安全监管部的学历情况：',df_count_degree['文化程度'][2],df_count_degree['数量'][2],'个，',
      df_count_degree['文化程度'][0],df_count_degree['数量'][0],'个，',
     df_count_degree['文化程度'][1],df_count_degree['数量'][1],'个。')

df_count_sex=df['性别'].value_counts()
print('安全监管部有',df_count_sex.index[0],df_count_sex['男'],'人，',df_count_sex.index[1],df_count_sex['女'],'人。')

df_count_enthic=df['民族'].value_counts()

print(df_count_enthic)

KeyError: '岗位名称'